In [ ]:
import pandas as pd

from datasets import load_dataset

# Charger tout le dataset (ou un plus grand sous-ensemble)
dataset = load_dataset("SetFit/amazon_reviews_multi_fr", split="train")
df = pd.DataFrame(dataset)
print(df['label'].value_counts())

label
0    40000
1    40000
2    40000
3    40000
4    40000
Name: count, dtype: int64


In [ ]:
from datasets import load_dataset
import pandas as pd

# Charger un plus grand sous-ensemble
dataset = load_dataset("SetFit/amazon_reviews_multi_fr", split="train[:100000]")
df = pd.DataFrame(dataset)

# Filtrer avis très négatifs
df_negatifs = df[df['label'] == 2].copy()
print(f"Nombre d'avis très négatifs : {len(df_negatifs)}")

Nombre d'avis très négatifs : 20000


In [ ]:
def generer_reponse(texte, sentiment="negative"):
    prompt = f"Le client a écrit : {texte}\nSentiment : {sentiment}\nRédige une réponse polie et professionnelle."
    # max_new_tokens limité pour accélérer
    return gen_pipe(prompt, max_new_tokens=100)[0]["generated_text"]


In [ ]:
from datasets import load_dataset
import pandas as pd
import re

# Charger dataset
dataset = load_dataset("SetFit/amazon_reviews_multi_fr", split="train[:1000]")  # 1000 pour test
df = pd.DataFrame(dataset)

# Fonction nettoyage simple
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zàâéèêëîïôùûç0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
df['texte_clean'] = df['text'].apply(clean_text)
df

,id,text,label,label_text,texte_clean
0,fr_0424335,A déconseiller - Article n'a fonctionné qu'une...,0,0,a déconseiller article na fonctionné quune foi...
1,fr_0452615,Si vous voulez être déçu achetez le produit ! ...,0,0,si vous voulez être déçu achetez le produit au...
2,fr_0407673,"Écran de mauvaise qualité, car il s'use en peu...",0,0,écran de mauvaise qualité car il suse en peu d...
3,fr_0579191,Cet engin ne sert à rien les sons sont pourris...,0,0,cet engin ne sert à rien les sons sont pourris...
4,fr_0931533,Très beau produit mais la grue n'a pas fonctio...,0,0,très beau produit mais la grue na pas fonction...
...,...,...,...,...,...
995,fr_0639004,"Suite à la livraison, le produit etait stocké ...",0,0,suite à la livraison le produit etait stocké d...
996,fr_0120006,Attention il ya la même sur eBay a 1 euros ave...,0,0,attention il ya la même sur ebay a 1 euros ave...
997,fr_0464706,Très déçu..le cutter est arrivé avec pièces ma...,0,0,très déçule cutter est arrivé avec pièces manq...
998,fr_0371594,Je commande un livre de chez Larousse je reçoi...,0,0,je commande un livre de chez larousse je reçoi...


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Tokenizer et modèle
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Pipeline
gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=120
)

# Fonction pour générer réponse
def generer_reponse(texte, sentiment):
    prompt = f"Le client a écrit : {texte}\nSentiment : {sentiment}\nRédige une réponse polie et professionnelle."
    return gen_pipe(prompt)[0]["generated_text"]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Depuis label
def label_sentiment(label):
    if label == 1:
        return "positive"
    elif label == 2:
        return "negative"
    else:
        return "neutral"

df['sentiment'] = df['label'].apply(label_sentiment)
df

,id,text,label,label_text,texte_clean,sentiment
0,fr_0424335,A déconseiller - Article n'a fonctionné qu'une...,0,0,a déconseiller article na fonctionné quune foi...,neutral
1,fr_0452615,Si vous voulez être déçu achetez le produit ! ...,0,0,si vous voulez être déçu achetez le produit au...,neutral
2,fr_0407673,"Écran de mauvaise qualité, car il s'use en peu...",0,0,écran de mauvaise qualité car il suse en peu d...,neutral
3,fr_0579191,Cet engin ne sert à rien les sons sont pourris...,0,0,cet engin ne sert à rien les sons sont pourris...,neutral
4,fr_0931533,Très beau produit mais la grue n'a pas fonctio...,0,0,très beau produit mais la grue na pas fonction...,neutral
...,...,...,...,...,...,...
995,fr_0639004,"Suite à la livraison, le produit etait stocké ...",0,0,suite à la livraison le produit etait stocké d...,neutral
996,fr_0120006,Attention il ya la même sur eBay a 1 euros ave...,0,0,attention il ya la même sur ebay a 1 euros ave...,neutral
997,fr_0464706,Très déçu..le cutter est arrivé avec pièces ma...,0,0,très déçule cutter est arrivé avec pièces manq...,neutral
998,fr_0371594,Je commande un livre de chez Larousse je reçoi...,0,0,je commande un livre de chez larousse je reçoi...,neutral


In [ ]:
from datasets import load_dataset
import pandas as pd

# Charger un plus grand sous-ensemble
dataset = load_dataset("SetFit/amazon_reviews_multi_fr", split="train[:100000]")
df = pd.DataFrame(dataset)

# Filtrer avis très négatifs
df_negatifs = df[df['label'] == 2].copy()
print(f"Nombre d'avis très négatifs : {len(df_negatifs)}")

Nombre d'avis très négatifs : 20000


In [ ]:
df

,id,text,label,label_text
0,fr_0424335,A déconseiller - Article n'a fonctionné qu'une...,0,0
1,fr_0452615,Si vous voulez être déçu achetez le produit ! ...,0,0
2,fr_0407673,"Écran de mauvaise qualité, car il s'use en peu...",0,0
3,fr_0579191,Cet engin ne sert à rien les sons sont pourris...,0,0
4,fr_0931533,Très beau produit mais la grue n'a pas fonctio...,0,0
...,...,...,...,...
99995,fr_0015825,Paquet endommagé par le transporteur Remplacem...,2,2
99996,fr_0574968,"produit pas chère, solide, efficace mais ... l...",2,2
99997,fr_0163691,"Pratique pour la barbe,mais nul pour les cheve...",2,2
99998,fr_0033738,"Livraison sans carton colis, dans l'emballage ...",2,2


In [ ]:

df_negatifs['texte_clean'] = df['text'].apply(clean_text)
df_negatifs

,id,text,label,label_text,texte_clean
80000,fr_0674869,Acheté pour remplacer un panier à ustensiles d...,2,2,acheté pour remplacer un panier à ustensiles d...
80001,fr_0879107,Vf annoncée sur le site et inexistante sur le DVD,2,2,vf annoncée sur le site et inexistante sur le dvd
80002,fr_0722436,Dommage que le produit avait un défaut de fabr...,2,2,dommage que le produit avait un défaut de fabr...
80003,fr_0354330,Leurres efficaces . J’ai ajouté une lumière no...,2,2,leurres efficaces jai ajouté une lumière noire...
80004,fr_0243976,J’avais une e-cigarette compacte et je voulais...,2,2,javais une ecigarette compacte et je voulais p...
...,...,...,...,...,...
99995,fr_0015825,Paquet endommagé par le transporteur Remplacem...,2,2,paquet endommagé par le transporteur remplacem...
99996,fr_0574968,"produit pas chère, solide, efficace mais ... l...",2,2,produit pas chère solide efficace mais les pou...
99997,fr_0163691,"Pratique pour la barbe,mais nul pour les cheve...",2,2,pratique pour la barbemais nul pour les cheveu...
99998,fr_0033738,"Livraison sans carton colis, dans l'emballage ...",2,2,livraison sans carton colis dans lemballage do...


In [ ]:
def generer_reponse(texte, sentiment="negative"):
    prompt = f"Le client a écrit : {texte}\nSentiment : {sentiment}\nRédige une réponse polie et professionnelle."
    # max_new_tokens limité pour accélérer
    return gen_pipe(prompt, max_new_tokens=100)[0]["generated_text"]


In [ ]:
# Limiter à 50 premiers avis négatifs pour test rapide
batch_size = 5  # petit batch pour vérifier rapidement
reponses = []

# On ne prend que les 25 premiers avis négatifs
df_test = df_negatifs.head(25)

for i in range(0, len(df_test), batch_size):
    batch = df_test['texte_clean'].iloc[i:i+batch_size]
    for texte in batch:
        reponses.append(generer_reponse(texte))
    print(f"Batch {i // batch_size + 1} / {len(df_test)//batch_size + 1} terminé")

# Ajouter la colonne au DataFrame
df_test['reponse'] = reponses

# Afficher pour vérifier
df_test[['texte_clean', 'reponse']]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batch 1 / 6 terminé


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batch 2 / 6 terminé


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Batch 4 / 6 terminé


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batch 5 / 6 terminé


/tmp/ipython-input-712077121.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['reponse'] = reponses


,texte_clean,reponse
80000,acheté pour remplacer un panier à ustensiles d...,Le client a écrit : acheté pour remplacer un p...
80001,vf annoncée sur le site et inexistante sur le dvd,Le client a écrit : vf annoncée sur le site et...
80002,dommage que le produit avait un défaut de fabr...,Le client a écrit : dommage que le produit ava...
80003,leurres efficaces jai ajouté une lumière noire...,Le client a écrit : leurres efficaces jai ajou...
80004,javais une ecigarette compacte et je voulais p...,Le client a écrit : javais une ecigarette comp...
80005,en fonction de lusage que lon en fait des imbé...,Le client a écrit : en fonction de lusage que ...
80006,certaines combinaisons de touches ne fonctionn...,Le client a écrit : certaines combinaisons de ...
80007,jai aimé la dimension ayant une malinoise,Le client a écrit : jai aimé la dimension ayan...
80008,le système de double seringue nest pas pratiqu...,Le client a écrit : le système de double serin...
80009,bonjour je maperçois au moment de me servir du...,Le client a écrit : bonjour je maperçois au mo...


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio


In [ ]:
!pip install pyngrok

In [ ]:

from pyngrok import ngrok

# colle ton authtoken ici :
ngrok.set_auth_token("P2QJ3K42DT2FFQ2FGG7OJIZ4OOXNJD3F")


In [ ]:
!ngrok config add-authtoken "35ZqMw5cB2BRKR7LZ3IdzHU0vrb_5g4ah9DGzr8iDW5eK3piV"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print("URL publique :", public_url)


URL publique : NgrokTunnel: "https://unelectrical-jacqui-reticently.ngrok-free.dev" -> "http://localhost:8000"


In [ ]:
from fastapi import FastAPI
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import re

# Simuler tes fonctions (ici juste un exemple minimal)
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zàâéèêëîïôùûç0-9\s]", "", text)
    return text

def analyse_sentiment(text):
    if "mauvais" in text or "pas bien" in text:
        return "NEGATIVE", 0.1
    else:
        return "POSITIVE", 0.9

def generer_reponse(sentiment):
    if sentiment == "POSITIVE":
        return "Merci pour votre avis positif !"
    else:
        return "Nous sommes désolés pour cette expérience."

# -----------------------------
#  Ton API FastAPI
# -----------------------------
app = FastAPI()

@app.get("/")
def home():
    return {"message": "API Avis Clients OK"}

@app.post("/analyse")
def analyser_avis(texte: str):
    texte_clean = clean_text(texte)
    sentiment, score = analyse_sentiment(texte_clean)
    reponse = generer_reponse(sentiment)
    return {"texte": texte_clean, "sentiment": sentiment, "reponse": reponse}

# -----------------------------
#  Lancer l'API sur Colab
# -----------------------------
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(" URL publique :", public_url)

uvicorn.run(app, port=8000)


 URL publique : NgrokTunnel: "https://unelectrical-jacqui-reticently.ngrok-free.dev" -> "http://localhost:8000"


RuntimeError: asyncio.run() cannot be called from a running event loop